# Check GPUs and Install libraries

In [1]:
#@title 1.1 Check GPU Status
import subprocess
simple_nvidia_smi_display = True#@param {type:"boolean"}
if simple_nvidia_smi_display:
  #!nvidia-smi
  nvidiasmi_output = subprocess.run(['nvidia-smi', '-L'], stdout=subprocess.PIPE).stdout.decode('utf-8')
  print(nvidiasmi_output)
else:
  #!nvidia-smi -i 0 -e 0
  nvidiasmi_output = subprocess.run(['nvidia-smi'], stdout=subprocess.PIPE).stdout.decode('utf-8')
  print(nvidiasmi_output)
  nvidiasmi_ecc_note = subprocess.run(['nvidia-smi', '-i', '0', '-e', '0'], stdout=subprocess.PIPE).stdout.decode('utf-8')
  print(nvidiasmi_ecc_note)

GPU 0: NVIDIA L4 (UUID: GPU-417126cd-a631-a8e0-0982-caf3b63f21a4)



# Imports

In [2]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch
from datasets import Dataset as HFDataset

from peft import LoraConfig, get_peft_model
from datasets import load_dataset

from transformers import (
    pipeline,
    AutoTokenizer,
    AutoModelForSequenceClassification,
    BertConfig,
    BertModel,
    Trainer,
    TrainingArguments,
    T5Tokenizer,
    T5ForConditionalGeneration,
    AutoTokenizer, 
    AutoModelForCausalLM,
    GPT2Model,
    GPT2Config,
    GPT2ForSequenceClassification,
    GPT2Tokenizer,
)
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Data pre-processing

In [3]:
train_hf_dataset = HFDataset.load_from_disk("SubMutantBench_train")
validation_hf_dataset = HFDataset.load_from_disk("SubMutantBench_validation")
test_hf_dataset = HFDataset.load_from_disk("SubMutantBench_test")
print(train_hf_dataset)
print(validation_hf_dataset)
print(test_hf_dataset)

Dataset({
    features: ['code1', 'code2', 'label'],
    num_rows: 1652
})
Dataset({
    features: ['code1', 'code2', 'label', '__index_level_0__'],
    num_rows: 825
})
Dataset({
    features: ['code1', 'code2', 'label', '__index_level_0__'],
    num_rows: 825
})


In [5]:
# Pre-process the original data
def preprocess_function(examples):
    inputs = ["code_1: " + examples["code1"][i] + " code_2: " + examples["code2"][i] for i in range(len(examples["code1"]))]
    targets = [i for i in examples["label"]]
    # targets = [str(i) for i in examples["label"]]
    
    max_length = 512
    return {"input_ids": tokenizer(inputs, truncation=True, padding="max_length", max_length=max_length)["input_ids"],
            "attention_mask": tokenizer(inputs, truncation=True, padding="max_length", max_length=max_length)["attention_mask"],
            "labels": torch.tensor(targets, dtype=torch.long),
            # "labels": tokenizer(targets, truncation=True, padding="max_length", max_length=256)["input_ids"]
        }

# Load the Tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

# Pre-process data
tokenized_train_dataset = train_hf_dataset.map(preprocess_function, batched=True, batch_size=4)
tokenized_validation_dataset = validation_hf_dataset.map(preprocess_function, batched=True, batch_size=4)
tokenized_test_dataset = test_hf_dataset.map(preprocess_function, batched=True, batch_size=4)

Map:   0%|          | 0/1652 [00:00<?, ? examples/s]

Map:   0%|          | 0/825 [00:00<?, ? examples/s]

Map:   0%|          | 0/825 [00:00<?, ? examples/s]

In [6]:
# Load pre-trained model
num_labels = 2
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=num_labels)

# Config LoRA specifications
lora_config = LoraConfig(
    # r=8,  # lower the rank
    r=32,  # lower the rank
    lora_alpha=32,
    # target_modules=["q", "v"],  # apply LoRA to q and v of attention modules
    # target_modules=["c_attn"],  # target query, key, and value together
    target_modules=["query", "value"],  # correct module names for BERT
    lora_dropout=0.1,
    bias="none",
    # task_type="SEQ_2_SEQ_LM",  # task type set to seq2seq generation
    task_type="SEQ_CLS",  # task type set to text classification
)

# Convert the model to LoRA model
model = get_peft_model(model, lora_config)

# Check the number of trainable parameters (for LoRA)
model.print_trainable_parameters()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 1,181,186 || all params: 109,492,996 || trainable%: 1.0788


In [7]:
# Adjust the batch size
batch_size = 4
training_args = TrainingArguments(
    output_dir="./results/lora-bert-emd",
    eval_strategy="epoch",
    learning_rate=5e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    metric_for_best_model="f1",
    weight_decay=0.01,
    num_train_epochs=6,
    save_strategy="no",
    logging_strategy="steps",
    logging_steps=10,
    report_to="mlflow",  # disable wandb etc.
    fp16=True,  # mixed precision training
    optim="adamw_torch",  # use torch original optimizer
)

def compute_metrics(pred):
    labels = pred.label_ids
    # Extract logits from the predictions & assuming logits are the first element in the tuple
    logits = pred.predictions[0] if isinstance(pred.predictions, tuple) else pred.predictions
    preds = logits.argmax(-1)
    f1 = f1_score(labels, preds, average="macro")
    acc = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average="macro")
    recall = recall_score(labels, preds, average="macro")
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_validation_dataset,
)

# Start Training
trainer.train()

# Save the model after LoRA fine-tuing
model.save_pretrained("./lora-bert-emd/model")
tokenizer.save_pretrained("./lora-bert-emd/tokenizer")

print("LoRA fine-tuning done, model saved!")

test_results = trainer.predict(tokenized_test_dataset)
predictions, labels, metrics = test_results
print(metrics)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.921200,0.394377,0.901818,0.948143,0.676000,0.733009
2,0.229400,0.461328,0.901818,0.948143,0.676000,0.733009
3,0.275000,0.384580,0.901818,0.948143,0.676000,0.733009
4,0.321400,0.367897,0.906667,0.950450,0.692000,0.751390
5,0.233000,0.361467,0.921212,0.957516,0.740000,0.802140
6,0.666400,0.370635,0.921212,0.957516,0.740000,0.802140


LoRA fine-tuning done, model saved!


{'test_loss': 0.32292336225509644, 'test_accuracy': 0.9321212121212121, 'test_precision': 0.9630118890356671, 'test_recall': 0.7741935483870968, 'test_f1': 0.8349622770919067, 'test_runtime': 4.4234, 'test_samples_per_second': 186.507, 'test_steps_per_second': 46.796}
